In [1]:
import h5py as hp
import numpy as np
import matplotlib.pyplot as plt
import illustris_python as il
import astro_utils.plots as auplt
from figrid.data_sort import DataSort
from figrid.data_container import DataContainer
from figrid.figrid import Figrid

from Corrfunc.theory import xi as cf

SIMPATH = '/Users/cosinga/sims/illustris/L75n1820TNG/'
auplt.RC()

In [2]:
h = 0.6774
box = 75.0
def get_halo_data(ss):
    data = {}
    f = il.groupcat.loadHalos(SIMPATH + 'output', ss, ['Group_M_Crit200', 'GroupPos', 'GroupFirstSub'])
    data['m200c'] = f['Group_M_Crit200'] * 1e10 / h
    data['x'] = f['GroupPos'] / 1e3
    out_mask = data['x'] >= 75
    data['x'][out_mask] = data['x'][out_mask] - 75
    data['cent_idx'] = f['GroupFirstSub']
    f = hp.File(SIMPATH + 'postprocessing/halo_structure_%03d.hdf5'%ss, 'r')
#     data['form'] = f['a_form'][:]
    data['c200c'] = f['c200c'][:]
    return data

def get_gal_data(ss):
    data = {}
    f = il.groupcat.loadSubhalos(SIMPATH + 'output', ss, ['SubhaloPos', 'SubhaloGrNr', 'SubhaloMassType', 'SubhaloStellarPhotometrics'])
    data['x'] = f['SubhaloPos'] / 1e3
    data['halo_idx'] = f['SubhaloGrNr']
    data['stmass'] = f['SubhaloMassType'][:, 4] * 1e10 / h
    data['gr'] = f['SubhaloStellarPhotometrics'][:, 4] - f['SubhaloStellarPhotometrics'][:, 5]
    return data

def get_hi_data(ss):
    
    return

In [3]:
def get_blue_mask(gr, ss):
    if ss == 99:
        return (gr <= 0.6)
    elif ss == 67:
        return gr <= 0.55
    elif ss == 50:
        return gr <= 0.5
    else:
        raise ValueError()
    
def get_res_mask(stmass):
    return stmass >= 2e8

In [4]:
hdatas = []; gdatas = []
for ss in [99, 67, 50]:
    print(ss)
    hdatas.append(get_halo_data(ss))
    gdatas.append(get_gal_data(ss))

99
67
50


In [5]:
# halo correlation function
import time
rbins = np.geomspace(0.5, (box - 1) / 2, 15, dtype = np.float64)
print(rbins)
samples = [0.001, 0.01, 0.1, 1]
for i in range(len(samples)):
    mass = hdatas[0]['m200c']
    mask = mass >= 2e9
    pos = hdatas[0]['x'][mask, :]
    out_mask = np.any((pos <= 0) | (pos >= box), axis = 1)
    pos = pos[~out_mask, :].astype(np.float64)
    nhalos = pos.shape[0]
    idxs = np.random.randint(0, nhalos, size = int(nhalos * samples[i]))
    print(samples[i], int(nhalos * samples[i]))
    start = time.time()
    res = cf(box, 4, rbins, pos[idxs, 0], pos[idxs, 1], pos[idxs, 2])
    end = time.time() - start
    print(end)

[ 0.5         0.679965    0.92470479  1.25753378  1.7101579   2.32569501
  3.1627824   4.30116263  5.84928006  7.95461138 10.81771458 14.71133448
 20.00638496 27.20728292 37.        ]
0.001 164984629489
0.014138936996459961
0.01 164984629489
0.0332338809967041
0.1 164984629489
1.7934379577636719
1 164984629489


Received signal = `Interrupt: 2' (signo = 2). Aborting 


RuntimeError: RuntimeError occurred

In [57]:
mask = pos >= box
print(pos[mask])
zmask = pos <= 0
print(pos[zmask])

[]
[0.]


In [64]:
N = 10000

nthreads = 4

seed = 42

np.random.seed(seed)

X = np.random.uniform(0, box, N)
X = X.astype(np.float32)
Y = np.random.uniform(0, box, N)
Y = Y.astype(np.float32)
Z = np.random.uniform(0, box, N)
Z = Z.astype(np.float32)

weights = np.ones_like(X)

# X = pos[:, 0].copy()
# Y = pos[:, 1].copy()
# Z = pos[:, 2].copy()
results = cf.xi(box, nthreads, rbins, X, Y, Z, output_ravg=True)

RuntimeError: RuntimeError occurred